In [ ]:
import zipfile
import requests

zip_file_path = 'tl_2020_24_tabblock20.zip'

# Open the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all files to the current working directory
    zip_ref.extractall()

shapefile_path = 'tl_2020_24_tabblock20.shp'
block_gdf = gpd.read_file(shapefile_path)

In [ ]:
url = "https://transit.land/api/v2/rest/feed_versions/8523f088e05dcd273b369dd4d65eb771c73c0a22/download?apikey=T2Jljq0EQqqe1JMfijkn3RdtVxoYpwuq"
response = requests.get(url)

if response.status_code == 200:
    with open("output.zip", "wb") as f:
        f.write(response.content)
    print("File downloaded successfully.")
else:
    print("Failed to download the file.")

import os

zip_file_path = 'output.zip'
extract_to_folder = 'transitland'
os.makedirs(extract_to_folder, exist_ok=True)

# Open the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all files to the current working directory
    zip_ref.extractall(extract_to_folder)

In [1]:
import pandas as pd
import geopandas as gpd
shapefile_path = 'tl_2020_24_tabblock20.shp'
block_gdf = gpd.read_file(shapefile_path)

/Users/yangxinyuxie/miniforge3/envs/aaforecast/lib/python3.11/site-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [5]:
block_gdf.geometry = block_gdf.geometry.centroid

#CODE FOR BLOCKS
compressed_csv_file_home = 'md_rac_S000_JT00_2020.csv.gz'
compressed_csv_file_work = 'md_wac_S000_JT00_2020.csv.gz'
gdf_filtered = block_gdf[block_gdf['COUNTYFP20'] == '510']

df_origins_blocks = pd.read_csv(compressed_csv_file_home)
df_destinations_blocks = pd.read_csv(compressed_csv_file_work)

df_origins_blocks['h_geocode'] = df_origins_blocks['h_geocode'].astype(str)
df_destinations_blocks['w_geocode'] = df_destinations_blocks['w_geocode'].astype(str)

df_origins_blocks = df_origins_blocks[(df_origins_blocks['h_geocode'].str[:5] == '24510') & (df_origins_blocks['C000'] != 0)]
df_destinations_blocks = df_destinations_blocks[(df_destinations_blocks['w_geocode'].str[:5] == '24510') & (df_destinations_blocks['C000'] != 0)]

df_origins_blocks['h_geocode'] = df_origins_blocks['h_geocode'].astype(int)
df_destinations_blocks['w_geocode'] = df_destinations_blocks['w_geocode'].astype(int)

gdf_filtered['GEOID20'] = gdf_filtered['GEOID20'].astype(int)
origins_merged_blocks = gdf_filtered.merge(df_origins_blocks, left_on = 'GEOID20', right_on = 'h_geocode')
destinations_merged_blocks = gdf_filtered.merge(df_destinations_blocks, left_on = 'GEOID20', right_on = 'w_geocode')

origins_merged_blocks['id'] = origins_merged_blocks.h_geocode
destinations_merged_blocks['id'] = destinations_merged_blocks.w_geocode

/var/folders/_y/kbgx8vrn4yjggv5d_bmd05c80000gn/T/ipykernel_10519/1147489711.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  block_gdf.geometry = block_gdf.geometry.centroid
/Users/yangxinyuxie/miniforge3/envs/aaforecast/lib/python3.11/site-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [2]:
%%time
# Allow 6 GB memory
import sys
sys.argv.append(["--max-memory", "6G"])

import datetime
from r5py import TransportNetwork, TravelTimeMatrixComputer, TransportMode

# # Filepath to OSM data
osm_fp = "maryland-latest.osm.pbf"

CPU times: user 2.21 s, sys: 1 s, total: 3.21 s
Wall time: 2.27 s


In [3]:
travel_time_matrix = []
zip_file_path = 'updated_gtfs/2023-04-07.zip'

transport_network_original = TransportNetwork(
    osm_fp,
    [
        zip_file_path
    ]
)

In [57]:
import r5py
import datetime
travel_time_matrix_computer = r5py.TravelTimeMatrixComputer(
    transport_network_original,
    origins=origins_merged_blocks[:5],
    destinations=destinations_merged_blocks[100:105],
    departure=datetime.datetime(2023,4,7,7,0),
    transport_modes=[TransportMode.TRANSIT, TransportMode.WALK],
    snap_to_network=True,
    departure_time_window = datetime.timedelta(hours=3)
)
travel_time_matrix_blocks = travel_time_matrix_computer.compute_travel_times()

In [58]:
gdf_filtered[gdf_filtered['GEOID20'] == 245102501032017].geometry

2354    POINT (-76.67780 39.26460)
Name: geometry, dtype: geometry

In [67]:
travel_time_matrix_blocks

,from_id,to_id,travel_time
0,245102704014006,245102704021002,19.0
1,245102704014006,245102703022006,20.0
2,245102704014006,245102501032017,NaN
3,245102704014006,245101502002006,102.0
4,245102704014006,245101102002005,81.0
5,245102720071010,245102704021002,NaN
6,245102720071010,245102703022006,106.0
7,245102720071010,245102501032017,NaN
8,245102720071010,245101502002006,74.0
9,245102720071010,245101102002005,97.0


In [12]:
travel_time_matrix_red = []

redline = 'redline.zip'
transport_network = r5py.TransportNetwork(
    osm_fp,
    [
        zip_file_path,
        redline,
    ]
)

In [60]:
travel_time_matrix_computer = r5py.TravelTimeMatrixComputer(
    transport_network,
    origins=origins_merged_blocks[:5],
    destinations=destinations_merged_blocks[100:105],
    departure=datetime.datetime(2023,4,7,7,0),
    transport_modes=[TransportMode.TRANSIT, TransportMode.WALK],
    snap_to_network=True,
    departure_time_window = datetime.timedelta(hours=2)
)
travel_time_matrix_blocks_red = travel_time_matrix_computer.compute_travel_times()
travel_time_matrix_red.append(travel_time_matrix_blocks)

In [62]:
travel_time_matrix_blocks_red

,from_id,to_id,travel_time
0,245102704014006,245102704021002,19.0
1,245102704014006,245102703022006,20.0
2,245102704014006,245102501032017,113.0
3,245102704014006,245101502002006,90.0
4,245102704014006,245101102002005,69.0
5,245102720071010,245102704021002,114.0
6,245102720071010,245102703022006,95.0
7,245102720071010,245102501032017,112.0
8,245102720071010,245101502002006,60.0
9,245102720071010,245101102002005,77.0


In [63]:
diff = travel_time_matrix_blocks['travel_time'] - travel_time_matrix_blocks_red['travel_time']

In [64]:
diff = diff[diff > 0]

In [65]:
diff

3     12.0
4     12.0
6     11.0
8     14.0
9     20.0
12    15.0
13    28.0
14    27.0
15     9.0
16    11.0
17    10.0
19     3.0
Name: travel_time, dtype: float64

In [66]:
diff.mean()

14.333333333333334

In [ ]:
diff.hist(bins = 50)

In [ ]:
import pandas as pd
import os
import urllib.request
import gzip
import shutil
from itertools import accumulate
import zipfile


def download_files(url, raw_path):
    zip_path, _ = urllib.request.urlretrieve(url)
    with zipfile.ZipFile(zip_path, "r") as f:
        f.extractall(raw_path)

def get_lodes_file(raw_path, lodes_type, file_name):
    lodes_file = raw_path + file_name
    if not os.path.isfile(lodes_file):
        print("Downloading LODES data into " + lodes_file)
        url = f"https://lehd.ces.census.gov/data/lodes/LODES7/md/{lodes_type}/{file_name}.gz"
        urllib.request.urlretrieve(url, raw_path + "tmp.gz")
        with gzip.open(raw_path + "tmp.gz", "rb") as f_in:
            with open(lodes_file, "wb") as f_out:
                shutil.copyfileobj(f_in, f_out)
        os.remove(raw_path + "tmp.gz")
    lodes_data = pd.read_csv(lodes_file)
    return lodes_data

def set_paths(prefix = "../"):
    processed_relpath = f"{prefix}processed_data/"
    raw_relpath = f"{prefix}raw_data/"

    processed_path = os.path.join(processed_relpath)
    raw_path = os.path.join(raw_relpath)
    return raw_path, processed_path

def replace_tracts(df):
    return df.replace(["24510180100", "24510180200"], "24510280600")

def restrict_to_Baltimore(df, col):
    tracts = pd.read_csv("../raw_data/Baltimore_tracts.csv")['tracts'].astype(str).tolist()
    return df[df[col].isin(tracts)]

def extract_tract_FIPS(df, col):
    return df[col].apply(lambda x: str(x)[-11:])

def FIPS_to_str(df, col):
    return df[col].apply(lambda x: str(x)[:11])


def cumulativeSum(lst):
    return list(accumulate(lst))